In [1]:
import sys
print(sys.version)

3.6.3 (default, Nov 13 2017, 08:48:07) 
[GCC 4.2.1 Compatible Apple LLVM 8.0.0 (clang-800.0.42.1)]


In [2]:
import gensim

I'm going to work with the Courtlistener Supreme Court opinions dataset, found at [https://www.courtlistener.com/api/bulk-data/opinions/scotus.tar.gz](https://www.courtlistener.com/api/bulk-data/opinions/scotus.tar.gz) --- they're a nonprofit without a lot of bandwidth, so leave a good half an hour to download it even on a fast connection --- which has been previously downloaded into supcourt and then unzipped into a bunch of individual case jsons in supcourt/scotus/ 

If you look at the data, you'll see that each JSON has a "html" field with a html-ized version of the opinion, as well as a "plain_text" field... but that the latter is sometimes empty.  So in order to extract texts, we'll try and get it out of plain_text first; if there's no plain text, we'll use beautiful soup to extract from html (or from several other alternate html fields)

In [13]:
from bs4 import BeautifulSoup
def extract_text(html):
    soup = BeautifulSoup(html, "lxml")
    for crap in soup(["script", "style", "meta"]):
        crap.extract()
    return soup.get_text()

In [14]:
# let's do an experiment and make sure this works right
import requests
response = requests.get("http://rulelaw.net")
print(extract_text(response.text))

The Rule of Law in the Real WorldThe Rule of Law in the Real WorldPaul Gowder"Paul Gowder's masterpiece articulates a new vision of the rule of the law that protects the disempowered and marginalized, and that demands that the nation-state rationalize its coercive power. He relentlessly attacks irrational social, economic and political hierarchies, particularly those that give continued vitality to racial inequality in the US today. As such his rule of law is firmly rooted in notions of human rights, and looks askance at soaring inequality in the US. This rule of law protects real people in need of real protection, rather than serving as just another rhetorical instrument in the arsenal of the wealthy and powerful. It also serves, incidentally, as a bedrock foundation for economic development and a tool that can help avoid financial disruptions like that seen in 2008."Steven A. Ramirez, author of Lawless Capitalism"The Rule of Law in the Real World explores and connects legal philosoph

Actually, this is ugly.  After the data finishes downloading, if cases come out this ugly too, I might break out jsoup or some other parser that does a better job getting text out of these things.